# Chapter 5: Fingerprinting-based Indoor Positioning

## Principles of Indoor Positioning and Indoor Navigation

---

### 📚 Learning Objectives

By the end of this notebook, you will be able to:

1. **Understand** the fingerprinting paradigm (offline training + online positioning)
2. **Implement** deterministic methods (Nearest Neighbor, k-Nearest Neighbor)
3. **Apply** probabilistic methods (Bayesian inference, MAP, Posterior Mean)
4. **Compare** different methods in terms of accuracy and speed
5. **Analyze** the impact of database density and measurement noise

### 📖 Book Reference

This notebook covers **Chapter 5: Fingerprinting-based Indoor Positioning** with:
- **Eq. (5.1)**: Nearest Neighbor (NN) positioning
- **Eq. (5.2)**: k-Nearest Neighbor (k-NN) weighted positioning
- **Eq. (5.3)-(5.5)**: Probabilistic/Bayesian methods (likelihood, MAP, posterior mean)

---


## 🚀 Setup (Google Colab)

**Set the `GITHUB_REPO` variable below to your repository URL, then run the setup cell.**

Example: `GITHUB_REPO = "https://github.com/YOUR_USERNAME/IPIN-Examples.git"`


In [ ]:
# ========================================
# IPIN Book Examples - Chapter 5: Fingerprinting
# ========================================

import os
import sys

# ============ CONFIGURATION ============
GITHUB_REPO = None  # Set your repo URL, e.g., "https://github.com/username/IPIN-Examples.git"
# =======================================

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    if os.path.exists('/content/IPIN-Examples/core'):
        os.chdir('/content/IPIN-Examples')
        print("✅ Repository already available.")
    elif GITHUB_REPO:
        print(f"📥 Cloning from {GITHUB_REPO}...")
        get_ipython().system(f'git clone {GITHUB_REPO}')
        os.chdir('/content/IPIN-Examples')
        get_ipython().system('pip install -e . -q')
        print("✅ Setup from GitHub complete!")
    else:
        print("❌ ERROR: GITHUB_REPO not set!")
        print("Please set GITHUB_REPO = 'https://github.com/YOUR_USERNAME/IPIN-Examples.git'")
        raise ValueError("GITHUB_REPO not configured.")
else:
    if os.path.basename(os.getcwd()) == 'notebooks':
        os.chdir('..')
    print(f"📂 Working directory: {os.getcwd()}")

# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

get_ipython().run_line_magic('matplotlib', 'inline')
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100

# Import fingerprinting modules
from core.fingerprinting import (
    load_fingerprint_database,
    nn_localize,
    knn_localize,
    fit_gaussian_naive_bayes,
    map_localize,
    posterior_mean_localize,
)

print("\n" + "="*60)
print("✅ Setup complete! Fingerprinting modules loaded.")
print("="*60)


---

# Part 1: Fingerprinting Overview

## 1.1 The Fingerprinting Paradigm

Fingerprinting uses a **pattern-matching** approach instead of geometric models:

### Offline Phase (Training)
1. Survey the environment at known **Reference Points (RPs)**
2. Record **RSS fingerprints** (signal strengths from multiple APs) at each RP
3. Build a **fingerprint database**: `{(location₁, fingerprint₁), (location₂, fingerprint₂), ...}`

### Online Phase (Positioning)
1. Measure current RSS fingerprint at unknown location
2. Compare against database using distance metrics
3. Estimate position based on best-matching reference points

## 1.2 Why Fingerprinting?

| Advantage | Description |
|-----------|-------------|
| **No propagation model** | Doesn't need path-loss exponent calibration |
| **Handles multipath** | Database captures actual signal behavior |
| **Works with any RF** | WiFi, BLE, cellular, etc. |

| Disadvantage | Description |
|--------------|-------------|
| **Site survey required** | Labor-intensive database collection |
| **Environment changes** | Database can become outdated |
| **Discrete locations** | Accuracy limited by RP density |


In [ ]:
# Load fingerprint database
print("="*70)
print("Loading Fingerprint Database")
print("="*70)

db_path = Path("data/sim/ch5_wifi_fingerprint_grid")
db = load_fingerprint_database(db_path)

print(f"\n📁 Database loaded from: {db_path}")
print(f"\n📊 Database Statistics:")
print(f"  Reference Points: {len(db.locations)}")
print(f"  Access Points:    {db.features.shape[1]}")
print(f"  Floors:           {db.floor_list}")
print(f"\n📍 Spatial Coverage:")
print(f"  X range: [{db.locations[:, 0].min():.1f}, {db.locations[:, 0].max():.1f}] m")
print(f"  Y range: [{db.locations[:, 1].min():.1f}, {db.locations[:, 1].max():.1f}] m")
print(f"\n📡 RSS Statistics:")
print(f"  Mean RSS:  {db.features.mean():.1f} dBm")
print(f"  RSS range: [{db.features.min():.1f}, {db.features.max():.1f}] dBm")


In [ ]:
# Visualize database
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Plot 1: Reference point locations
ax1 = axes[0]
floor_id = 0  # Show floor 0
floor_mask = db.get_floor_mask(floor_id)
ax1.scatter(db.locations[floor_mask, 0], db.locations[floor_mask, 1], 
            c='blue', marker='s', s=60, alpha=0.7, label='Reference Points')
ax1.set_xlabel('X (m)', fontsize=12)
ax1.set_ylabel('Y (m)', fontsize=12)
ax1.set_title(f'Reference Point Layout (Floor {floor_id})', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_aspect('equal')

# Plot 2: RSS heatmap for first AP
ax2 = axes[1]
floor_locs = db.locations[floor_mask]
floor_rss = db.features[floor_mask, 0]  # First AP
scatter = ax2.scatter(floor_locs[:, 0], floor_locs[:, 1], 
                       c=floor_rss, cmap='RdYlGn', s=60, alpha=0.8)
cbar = plt.colorbar(scatter, ax=ax2)
cbar.set_label('RSS (dBm)', fontsize=11)
ax2.set_xlabel('X (m)', fontsize=12)
ax2.set_ylabel('Y (m)', fontsize=12)
ax2.set_title('RSS Fingerprint (AP #1)', fontsize=14, fontweight='bold')
ax2.set_aspect('equal')

plt.tight_layout()
plt.show()

print("\n💡 Observation: RSS varies spatially - this pattern enables positioning!")


---

# Part 2: Deterministic Methods

## 2.1 Nearest Neighbor (NN) - Eq. 5.1

Find the reference point with the most similar fingerprint:

$$i^* = \arg\min_i D(\mathbf{z}, \mathbf{f}_i)$$

$$\hat{\mathbf{x}} = \mathbf{x}_{i^*}$$

where:
- $\mathbf{z}$ = query fingerprint (measured RSS values)
- $\mathbf{f}_i$ = database fingerprint at reference point $i$
- $D(\cdot, \cdot)$ = distance metric (Euclidean, Manhattan, etc.)

## 2.2 k-Nearest Neighbor (k-NN) - Eq. 5.2

Average the positions of the k closest reference points:

$$\hat{\mathbf{x}} = \frac{\sum_{i \in \mathcal{N}_k} w_i \mathbf{x}_i}{\sum_{i \in \mathcal{N}_k} w_i}$$

where:
- $\mathcal{N}_k$ = set of k nearest neighbors
- $w_i$ = weight (uniform: $w_i=1$, or inverse distance: $w_i = 1/D_i$)


In [ ]:
# Generate test queries by interpolating from database
def generate_test_queries(db, n_queries, floor_id, noise_std=0.0, seed=42):
    """Generate test fingerprints at random locations."""
    np.random.seed(seed)
    
    floor_mask = db.get_floor_mask(floor_id)
    rp_locs = db.locations[floor_mask]
    rp_features = db.features[floor_mask]
    
    # Random locations within bounding box
    min_x, max_x = rp_locs[:, 0].min(), rp_locs[:, 0].max()
    min_y, max_y = rp_locs[:, 1].min(), rp_locs[:, 1].max()
    
    true_locs = np.column_stack([
        np.random.uniform(min_x + 2, max_x - 2, n_queries),
        np.random.uniform(min_y + 2, max_y - 2, n_queries),
    ])
    
    # Interpolate fingerprints from nearby RPs
    queries = []
    for loc in true_locs:
        dists = np.linalg.norm(rp_locs - loc, axis=1)
        k_nearest = min(4, len(dists))
        nearest_idx = np.argpartition(dists, k_nearest)[:k_nearest]
        
        weights = 1.0 / (dists[nearest_idx] + 1e-3)
        weights /= weights.sum()
        
        query_fp = np.sum(weights[:, None] * rp_features[nearest_idx], axis=0)
        query_fp += np.random.randn(len(query_fp)) * noise_std
        queries.append(query_fp)
    
    return np.array(queries), true_locs

# Generate test data
n_queries = 50
noise_std = 2.0  # 2 dBm measurement noise
queries, true_locs = generate_test_queries(db, n_queries, floor_id, noise_std)

print(f"Generated {n_queries} test queries with {noise_std} dBm noise")
print(f"Query fingerprint shape: {queries.shape}")


In [ ]:
# Example 1: Nearest Neighbor (NN) Positioning
print("="*70)
print("Example 1: Nearest Neighbor (NN) Positioning - Eq. 5.1")
print("="*70)

nn_errors = []
nn_estimates = []

for query, true_loc in zip(queries, true_locs):
    est_loc = nn_localize(query, db, metric="euclidean", floor_id=floor_id)
    nn_estimates.append(est_loc)
    nn_errors.append(np.linalg.norm(est_loc - true_loc))

nn_errors = np.array(nn_errors)
nn_estimates = np.array(nn_estimates)

print(f"\n📊 NN Results:")
print(f"  RMSE:            {np.sqrt(np.mean(nn_errors**2)):.2f} m")
print(f"  Mean error:      {np.mean(nn_errors):.2f} m")
print(f"  Median error:    {np.median(nn_errors):.2f} m")
print(f"  90th percentile: {np.percentile(nn_errors, 90):.2f} m")


In [ ]:
# Example 2: k-Nearest Neighbor (k-NN) Positioning
print("="*70)
print("Example 2: k-Nearest Neighbor (k-NN) Positioning - Eq. 5.2")
print("="*70)

# Test different values of k
k_values = [1, 3, 5, 7]
knn_results = {}

for k in k_values:
    knn_errors = []
    knn_estimates = []
    
    for query, true_loc in zip(queries, true_locs):
        est_loc = knn_localize(query, db, k=k, metric="euclidean", 
                               weighting="inverse_distance", floor_id=floor_id)
        knn_estimates.append(est_loc)
        knn_errors.append(np.linalg.norm(est_loc - true_loc))
    
    knn_results[k] = {
        'errors': np.array(knn_errors),
        'estimates': np.array(knn_estimates),
        'rmse': np.sqrt(np.mean(np.array(knn_errors)**2)),
    }

print(f"\n📊 k-NN Results (inverse distance weighting):")
print(f"\n{'k':<5} {'RMSE (m)':<12} {'Median (m)':<12} {'P90 (m)':<12}")
print("-" * 41)
for k, res in knn_results.items():
    rmse = res['rmse']
    median = np.median(res['errors'])
    p90 = np.percentile(res['errors'], 90)
    print(f"{k:<5} {rmse:<12.2f} {median:<12.2f} {p90:<12.2f}")

# Best k
best_k = min(knn_results, key=lambda k: knn_results[k]['rmse'])
print(f"\n✅ Best k = {best_k} (RMSE = {knn_results[best_k]['rmse']:.2f} m)")


---

# Part 3: Probabilistic Methods

## 3.1 Bayesian Fingerprinting (Eqs. 5.3-5.5)

Instead of finding the "nearest" fingerprint, compute the **probability** of each location given the observation:

### Likelihood (Eq. 5.3)
Assuming Gaussian Naive Bayes (each AP independent):

$$p(\mathbf{z} | \mathbf{x}_i) = \prod_{j=1}^{M} \mathcal{N}(z_j; \mu_{ij}, \sigma_{ij}^2)$$

### MAP Estimate (Eq. 5.4)
$$\hat{\mathbf{x}}_{MAP} = \arg\max_i p(\mathbf{x}_i | \mathbf{z}) = \arg\max_i p(\mathbf{z} | \mathbf{x}_i) p(\mathbf{x}_i)$$

### Posterior Mean (Eq. 5.5)
$$\hat{\mathbf{x}}_{mean} = \sum_i p(\mathbf{x}_i | \mathbf{z}) \mathbf{x}_i$$

**Advantage**: Provides uncertainty quantification and smoother estimates


In [ ]:
# Example 3: Probabilistic Fingerprinting
print("="*70)
print("Example 3: Probabilistic Fingerprinting - Eqs. 5.3-5.5")
print("="*70)

# Fit Gaussian Naive Bayes model
print("\n📈 Fitting Gaussian Naive Bayes model...")
model = fit_gaussian_naive_bayes(db, min_std=2.0)
print("   Model fitted!")

# MAP positioning
print("\n🎯 MAP Positioning (Eq. 5.4):")
map_errors = []
for query, true_loc in zip(queries, true_locs):
    est_loc = map_localize(query, model, floor_id=floor_id)
    map_errors.append(np.linalg.norm(est_loc - true_loc))
map_errors = np.array(map_errors)

print(f"  RMSE:   {np.sqrt(np.mean(map_errors**2)):.2f} m")
print(f"  Median: {np.median(map_errors):.2f} m")

# Posterior Mean positioning
print("\n🎯 Posterior Mean Positioning (Eq. 5.5):")
pm_errors = []
for query, true_loc in zip(queries, true_locs):
    est_loc = posterior_mean_localize(query, model, floor_id=floor_id)
    pm_errors.append(np.linalg.norm(est_loc - true_loc))
pm_errors = np.array(pm_errors)

print(f"  RMSE:   {np.sqrt(np.mean(pm_errors**2)):.2f} m")
print(f"  Median: {np.median(pm_errors):.2f} m")

print("\n💡 Note: Posterior Mean often provides smoother estimates than MAP")


---

# Part 4: Method Comparison


In [ ]:
# Comprehensive comparison visualization
print("="*70)
print("Method Comparison")
print("="*70)

# Collect all results
all_results = {
    'NN': nn_errors,
    'k-NN (k=3)': knn_results[3]['errors'],
    'k-NN (k=5)': knn_results[5]['errors'],
    'MAP': map_errors,
    'Posterior Mean': pm_errors,
}

# Summary table
print(f"\n📊 Results Summary:\n")
print(f"{'Method':<18} {'RMSE (m)':<12} {'Median (m)':<12} {'P90 (m)':<12}")
print("-" * 54)
for method, errors in all_results.items():
    rmse = np.sqrt(np.mean(errors**2))
    median = np.median(errors)
    p90 = np.percentile(errors, 90)
    print(f"{method:<18} {rmse:<12.2f} {median:<12.2f} {p90:<12.2f}")

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot 1: Box plot comparison
ax1 = axes[0]
colors = ['#3498db', '#e74c3c', '#2ecc71', '#9b59b6', '#f39c12']
bp = ax1.boxplot([all_results[m] for m in all_results], 
                  labels=list(all_results.keys()), patch_artist=True)
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)
ax1.set_ylabel('Position Error (m)', fontsize=12)
ax1.set_title('Error Distribution by Method', fontsize=14, fontweight='bold')
ax1.tick_params(axis='x', rotation=30)
ax1.grid(True, alpha=0.3, axis='y')

# Plot 2: CDF comparison
ax2 = axes[1]
for (method, errors), color in zip(all_results.items(), colors):
    sorted_errors = np.sort(errors)
    cdf = np.arange(1, len(sorted_errors) + 1) / len(sorted_errors)
    ax2.plot(sorted_errors, cdf, label=method, color=color, linewidth=2)
ax2.axhline(y=0.90, color='gray', linestyle='--', alpha=0.5)
ax2.set_xlabel('Position Error (m)', fontsize=12)
ax2.set_ylabel('CDF', fontsize=12)
ax2.set_title('Cumulative Distribution Function', fontsize=14, fontweight='bold')
ax2.legend(fontsize=9)
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0, None)

# Plot 3: Spatial visualization (NN vs k-NN)
ax3 = axes[2]
# Show reference points
floor_mask = db.get_floor_mask(floor_id)
ax3.scatter(db.locations[floor_mask, 0], db.locations[floor_mask, 1],
            c='lightgray', marker='s', s=30, alpha=0.5, label='Reference Points')
# Show true vs estimated for first 20 queries
for i in range(min(20, len(true_locs))):
    ax3.plot([true_locs[i, 0], nn_estimates[i, 0]], 
             [true_locs[i, 1], nn_estimates[i, 1]], 
             'r-', alpha=0.3, linewidth=1)
ax3.scatter(true_locs[:20, 0], true_locs[:20, 1], c='green', marker='o', 
            s=50, label='True Location', zorder=5)
ax3.scatter(nn_estimates[:20, 0], nn_estimates[:20, 1], c='red', marker='x', 
            s=50, label='NN Estimate', zorder=5)
ax3.set_xlabel('X (m)', fontsize=12)
ax3.set_ylabel('Y (m)', fontsize=12)
ax3.set_title('NN Positioning Results (sample)', fontsize=14, fontweight='bold')
ax3.legend(fontsize=9)
ax3.set_aspect('equal')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


---

# Summary

## Key Takeaways

### 1. Method Categories

| Category | Methods | Characteristics |
|----------|---------|-----------------|
| **Deterministic** | NN, k-NN | Fast, simple, discrete outputs |
| **Probabilistic** | MAP, Posterior Mean | Uncertainty quantification, smoother |

### 2. Method Selection Guide

| Scenario | Recommended | Reason |
|----------|-------------|--------|
| Real-time, dense RPs | **k-NN (k=3-5)** | Fast, good accuracy |
| Sparse RPs | **Posterior Mean** | Better interpolation |
| Need uncertainty | **Probabilistic** | Provides confidence |
| Maximum speed | **NN** | Single lookup |

### 3. Key Findings

1. **k-NN often outperforms NN** by averaging multiple neighbors
2. **Inverse distance weighting** better than uniform weights
3. **Optimal k** depends on RP density (typically k=3-5)
4. **Probabilistic methods** provide smoother estimates but are slower

## Practical Considerations

- **Database quality**: More RPs → better accuracy but higher survey cost
- **Environment changes**: Furniture, people affect RSS → database may need updates
- **Multi-floor**: Floor detection is critical before horizontal positioning

---

## Exercises

1. **Vary noise level**: How does 1 dBm vs 5 dBm noise affect accuracy?
2. **Try sparse database**: Load `ch5_wifi_fingerprint_sparse` - how much accuracy is lost?
3. **Weighting schemes**: Compare uniform vs inverse-distance weighting for k-NN

---

**Next Steps:** Chapter 6 (Dead Reckoning) for sensor-based trajectory estimation!
